# Version pour travail automatique depuis lemmatisation avec Analog.

In [1]:
def d_in_pos(csv_lu):
    
    """
    Fonction récupérant le contenu d'un CSV avec les colones suivantes :
        - "Numero de token"
        - "Mot forme"
        - "Modifié"
        - "Lemme(s)"
        - "POS"
        
    Cette fonction boucle sur chaque token et chaque fois garde un trio de tokens
    dont les numéros se suivent (en excluant la ponctuation). Puis, sur ce trio, on
    teste des conditions dont la principale est qu'Analog a hésité entre une forme de
    déterminant et une forme de pronom (mais pas "S") et que le mot suivant n'est pas
    un nom commun.
    
    La fonction ajoute alors la nouvelle forle dictionnaire du token comme valeur.
    Une fois tous les tokens analysés, la fonction retourne le dictionnaire contenant
    tous les tokens modifiés.
    
    :param csv_lu: Le contenu du fichier CSV sur lequel on souhaite
            effectuer des vérifications, extrait et lu par la fonction principale.
            
    :return: Retourne un dictionnaire dont les clés sont les numéros des tokens
            contenus et les valeurs la ligne entière du CSV modifiée par la fonction.
    
    """
    
    # Création de dictionnaires locaux, vides.
    dico_mots = {}
    dico_modifies = {}
    # Compteur de tokens modifiés, compteur du total de tokens.
    compteur = 0
    total = 0
    # Liste des valeurs PoS correspondant à des déterminants.
    D_pos = ['Da', 'Dd', 'Ds', 'Dn', 'Dp', 'Di', 'Dr', 'Dt']
    # Liste des valeurs PoS correspondant à de la ponctuation.
    ponct_pos = ['Fs', 'Fw', 'Fo']
    # Liste destinée à rassembler les valeurs PoS à enlever d'une liste de PoS
    # le cas échéant.
    a_enlever = []
    separateur = "|"
    
    # On boucle sur les lignes du CSV d'entrée pour créer
    # un dictionnaire général (dico_mots) à exploiter.
    for row in csv_lu:
            
        # On stocke l'identifiant du token, en tant qu'integer, dans une variable.
        identifiant = int(row['ID'])
        
        # On ajoute l'unité au dictionnaire général, avec l'identifiant comme clé
        # et on ajoute 1 au total de tokens.
        dico_mots[identifiant] = row
        total += 1
        
    # On boucle sur les entrées du dictionnaire.
    for identifiant in dico_mots.keys():
        
        # S'il y a encore au moins 2 tokens après le token actuel, on continue.
        if identifiant + 2 in dico_mots.keys():
            
            # On crée trois variables sur trois mots qui se suivent.
            # Le mot n°2 sera le mot-cible ici, les deux autres
            # représentant son contexte.
            # À la fin des tests, le mot_cible uniquement sera
            # susceptible d'être modifié.
            
            mot_precedent = dico_mots[identifiant]
            mot_cible = dico_mots[identifiant + 1]
            mot_suivant = dico_mots[identifiant + 2]
            
            
            # Si le mot-cible a plusieurs PoS possibles, dont une forme de déterminant,
            # et que le mot suivant est un nom commun, on continue.
            if "D" in mot_cible['POS'] and "|" in mot_cible['POS'] and mot_suivant['POS'] == "Nc":
                
                # Le token sera modifié, donc on ajoute 1 au compteur de mots modifiés.
                compteur += 1
        
                # On crée des listes pour les PoS et les lemmes, pour pouvoir comparer les
                # différentes valeurs séparément aux listes déclarées au début de la fonction.
                liste_pos_2 = mot_cible['POS'].split("|")
                liste_lemmes = mot_cible["LEMMES"].split("|")
                
                # Cas particulier : le mot-cible a "LEDIT" comme lemme :
                # il prendra "Dd" comme PoS.
                if mot_cible['LEMMES'] == "LEDIT":
                    mot_cible['POS'] = "Dd"
            
                # Sinon : si le mot précédent n'est pas strictement un déterminant
                # ou c'est de la ponctuation : on récupère la valeur PoS du mot-cible
                # correspondant à un déterminant et le lemme qui va avec.
                elif mot_precedent['POS'] not in D_pos or mot_precedent['POS'] in ponct_pos:
                    for valeur in liste_pos_2:
                        if valeur in D_pos:
                            mot_cible['POS'] = valeur
                            if "|" in mot_cible['LEMMES']:
                                index_pos = liste_pos_2.index(valeur)
                                mot_cible["Lemme(s)"] = liste_lemmes[index_pos]
        
                # Sinon : si le mot-cible a "Ag" parmi ses valeurs de PoS possibles,
                # c'est son PoS et on prend le lemme qui lui correspond.
                elif "Ag" in liste_pos_2:
                    mot_cible['POS'] = 'Ag'
                    if "|" in mot_cible['LEMMES']:
                        index_pos = liste_pos_2.index("Ag")
                        mot_cible["Lemme(s)"] = liste_lemmes[index_pos]
         
                # Sinon : si le mot-cible a "As" parmi ses valeurs de PoS possibles,
                # c'est son PoS et on prend le lemme qui lui correspond.
                elif "As" in liste_pos_2:
                    mot_cible['POS'] = 'As'
                    if "|" in mot_cible['LEMMES']:
                        index_pos = liste_pos_2.index("As")
                        mot_cible["Lemme(s)"] = liste_lemmes[index_pos]
                        
                        
                # Si toutes les autres options ont échoué,
                # on enlève de la liste de PoS possibles du mot-cible toute
                # valeur correspondant à un déterminant et les lemmes qui vont avec.
                else:
                    for valeur in liste_pos_2:
                        if valeur in D_pos:
                            a_enlever.append(valeur)
                
                    for valeur in a_enlever:
                        if "|" in mot_cible['LEMMES']:
                            index_pos = liste_pos_2.index(valeur)
                            lemme_en_moins = liste_lemmes[index_pos]
                            liste_lemmes.remove(lemme_en_moins)
                
                        liste_pos_2.remove(valeur)
                    
                    mot_cible['LEMMES'] = separateur.join(liste_lemmes)
                    mot_cible['POS'] = separateur.join(liste_pos_2)
                    a_enlever = []
                
                # On ajoute le token modifié au dictionnaire retourné par la fonction.
                dico_modifies[mot_cible['ID']] = mot_cible
                
                
    # On calcule les totaux et pourcentages de tokens modifiés par la fonction.
    pourcent_traites = compteur * 100 / total
    print(compteur, ":", round(pourcent_traites, 1), "%")
    
    # On retourne les tokens modifiés.
    return dico_modifies

In [2]:
def desambiguiser(csv_entree, csv_sortie):
    
    """
        Fonction récupérant un CSV dont chaque ligne est un token lemmatisé,
        avec les colonnes telles quelles :
        
            - "Numero de token" --- étant le numéro constituant la valeur du @n
                    de l'élément <w> dans le XML dont les tokens ont été extraits
                    (en principe, chaque token a un numéro unique).
                    
            - "Mot forme" --- étant le texte de l'élément <w> original, avec
                    abréviations résolues et graphie modernisée s'il y a lieu.
                    
            - "Modifié" --- contient soit "yes", soit "no" ; la valeur a été produite
                    par le script précédent (NV n°4 : désambiguïsation simple) et
                    indique si les valeurs de la ligne sont exactement celles produites
                    par Analog ("no") ou si le script précédent les a modifiées ("yes").
                    
            - "Lemme(s)" --- étant le lemme préconisé par Analog, ou bien les lemmes
                    entre lesquels Analog hésite, séparés par des '|'.
                    
            - "POS" --- étant la valeur POS préconisée par Analog, ou bien les
                    différentes valeurs entre lesquelles Analog hésite, séparées par
                    des '|'.
        
        La fonction crée à partir du CSV d'entrée un dictionnaire qu'elle transmet à la
        fonction secondaire chargée d'analyser les tokens et les modifier si besoin et
        récupère un dictionnaire contenant uniquement les tokens modifiés.
        
        Puis elle produit un autre fichier CSV. Si le token se trouve dans le dictionnaire
        retourné par la fonction secondaire, c'est cette version qui sera écrite dans le
        nouveau CSV. Sinon, la version originale sera copiée.
        
        :param csv_entree: Le chemin interne du fichier CSV sur lequel on souhaite
            effectuer des vérifications.
            
        :param csv_sortie: Le chemin interne du fichier CSV dans lequel écrire
            les tokens modifiés.
            
    """
        
    import csv
    colonnes = [
            'ID',
            'TOKEN',
            'Modifié',
            'LEMMES',
            'POS'
        ]
    row_modifie = {}
    
    # Ouvrir le CSV d'Analog et le lire.
    with open(csv_entree) as csv_a_lire:
        csv_lu = csv.DictReader(csv_a_lire)
        
        # Récupérer la liste des tokens contenant D dans le POS
        # et modifiés par la fonction.
        # La fonction retourne un dictionnaire ne contenant que
        # les nouvelles formes des tokens modifiés.
        dico_modifies = d_in_pos(csv_lu)
        
        # ATTENTION :
        # type(row['ID']) = str
    
    # Rouvrir le CSV d'entrée et le lire.
    with open(csv_entree) as csv_a_lire:
        csv_lu = csv.DictReader(csv_a_lire)

        # Ouvrir le fichier de sortie et le lire en mode écriture.
        with open(csv_sortie, 'w') as csv_a_ecrire:
            a_ecrire = csv.DictWriter(csv_a_ecrire, fieldnames = colonnes)
            # Écrire les noms des colonnes.
            a_ecrire.writeheader()
            
            # Boucler à nouveau sur les lignes du CSV d'entrée.
            for row in csv_lu:
                
                # Si le token se trouve dans les formes modifiées,
                # on écrit la nouvelle forme dans le nouveau CSV.
                if row['ID'] in dico_modifies.keys():
                    row_modifie = dico_modifies[row['ID']]
                    
                    a_ecrire.writerow({
                        'ID' : row_modifie['ID'],
                        'TOKEN' : row_modifie['TOKEN'],
                        'Modifié' : "yes",
                        'LEMMES' : row_modifie['LEMMES'],
                        'POS' : row_modifie['POS']
                    })
                    row_modifie = {}
                
                # Si le token n'était pas dans les formes modifiées,
                # on écrit la forme originelle.
                else:
                    a_ecrire.writerow({
                        'ID' : row['ID'],
                        'TOKEN' : row['TOKEN'],
                        'Modifié' : row['Modifié'],
                        'LEMMES' : row['LEMMES'],
                        'POS' : row['POS']
                    })

In [3]:
desambiguiser(
    '/home/erminea/Documents/CONDE/GC/2021-05-27_GC_tokens_xe_fait.csv',
    '/home/erminea/Documents/CONDE/GC/2021-05-27_GC_tokens_det_fait.csv'
)

785 : 1.3 %


In [5]:


desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Basnage-xe-fait.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Basnage-det-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Berault-xe-fait.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Berault-det-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/instructions-xe-fait.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/instructions-det-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Merville-xe-fait.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Merville-det-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Pesnelle-xe-fait.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Pesnelle-det-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Rouille-xe-fait.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Rouille-det-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/ruines-xe-fait.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/ruines-det-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/tac-xe-fait.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/tac-det-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Terrien-xe-fait.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Terrien-det-fait.csv'
)

3051 : 0.2 %
644 : 0.1 %
64 : 0.6 %
1547 : 0.3 %
2261 : 0.3 %
1202 : 0.4 %
0 : 0.0 %
0 : 0.0 %
138 : 0.0 %
